In [48]:
import numpy as np
import pandas as pd
from src.prepare_dataset import prepare_data
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [9]:
prepare_data("data/final_dataset.csv", "prep_data")

data/final_dataset.csv


2024-07-26 18:21:29.075 | INFO     | src.prepare_dataset:prepare_data:31 - Dropped the unnecessary columns
2024-07-26 18:21:29.076 | INFO     | src.prepare_dataset:prepare_data:36 - Dataset size BEFORE dropping the rows by date: 372084
2024-07-26 18:21:29.815 | INFO     | src.prepare_dataset:prepare_data:38 - Dataset size AFTER dropping the rows by date: 369697
2024-07-26 18:21:30.228 | INFO     | src.prepare_dataset:prepare_data:56 - Overall encoded 5 cateforical features
2024-07-26 18:21:30.228 | INFO     | src.prepare_dataset:prepare_data:57 - ['school_hours', 'power_on', 'classroom_type', 'school_day', 'break_time']
2024-07-26 18:21:30.873 | INFO     | src.prepare_dataset:prepare_data:79 - Filled the missing values of variables
2024-07-26 18:21:30.880 | INFO     | src.prepare_dataset:prepare_data:82 - Sliced the data into train, valid, test splits with the proportion: 
2024-07-26 18:21:30.937 | INFO     | src.prepare_dataset:prepare_data:85 - Applied the MinMaxScaler to the dataset

In [11]:
train = pd.read_csv("prep_data/train.csv")
val = pd.read_csv("prep_data/valid.csv")
test = pd.read_csv("prep_data/test.csv")

In [24]:
def rearrange(data):
    features = data.drop(['measured_t', 'tmstamp'], axis=1)
    target = data['measured_t']
    combined = pd.concat([features, target], axis=1)
    return combined

In [25]:
train = rearrange(train)
val = rearrange(val)
test = rearrange(test)

In [27]:
train.head()

,classroom_type,school_day,school_hours,break_time,power_on,trackerwm_avg,shadowwm_avg,sunwm_avg,shadowbandwm_avg,dnicalc_avg,...,winddir_d1_wvt,winddir_sd1_wvt,bp_mb_avg,uva_avg,uvb_avg,Month,Day,Hour,Minute,measured_t
0,0,1,0,0,1,0.000000,0.764375,0.010247,0.061300,0.282270,...,0.000000,0.748538,0.106834,0.008796,0.074619,3,16,7,17,0.621711
1,1,1,0,0,1,0.000000,0.764626,0.010425,0.061711,0.282342,...,0.000000,0.748538,0.106678,0.008796,0.074433,3,16,7,18,0.631579
2,1,1,0,0,1,0.000000,0.764751,0.010425,0.061754,0.282256,...,0.000000,0.748538,0.105712,0.008642,0.074248,3,16,7,19,0.644737
3,0,1,0,0,1,0.000730,0.766587,0.011724,0.064445,0.282355,...,0.005905,0.764025,0.107146,0.008488,0.074248,3,16,7,27,0.628289
4,1,1,0,0,1,0.000332,0.774045,0.016519,0.079077,0.282305,...,0.000000,0.748538,0.106023,0.008951,0.074248,3,16,7,33,0.611842


In [20]:
def create_sequences(data, seq_length):
    xs = []
    ys = []
    for i in range(len(data)-seq_length):
        x = data.iloc[i:(i+seq_length), :-1].values
        y = data.iloc[i+seq_length, -1]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

In [28]:
seq_length = 10
X_train, y_train = create_sequences(train, seq_length)
X_test, y_test = create_sequences(test, seq_length)
X_val, y_val = create_sequences(val, seq_length)

In [39]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])
        return out

In [43]:
# Hyperparameters
input_size = X_train.shape[2]
hidden_size = 50
num_layers = 2
output_size = 1
num_epochs = 10
batch_size = 32
learning_rate = 0.001

In [41]:
# Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [42]:
# Initialize the model, loss function and optimizer
device = torch.device('cpu')
model = RNN(input_size, hidden_size, num_layers, output_size).to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [44]:
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()    

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

/Users/Sruthi/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
/Users/Sruthi/anaconda3/lib/python3.7/site-packages/torch/nn/modules/loss.py:101: UserWarning: Using a target size (torch.Size([25])) that is different to the input size (torch.Size([25, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)


Epoch [1/10], Loss: 0.0781
Epoch [2/10], Loss: 0.0897
Epoch [3/10], Loss: 0.0827
Epoch [4/10], Loss: 0.1031
Epoch [5/10], Loss: 0.1138
Epoch [6/10], Loss: 0.0971
Epoch [7/10], Loss: 0.0655
Epoch [8/10], Loss: 0.0885
Epoch [9/10], Loss: 0.0892
Epoch [10/10], Loss: 0.0737


In [45]:
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)

In [46]:
model.eval()
with torch.no_grad():
    predictions = model(X_val_tensor).cpu().numpy()

In [51]:
np.mean(np.abs(predictions - y_val_tensor))

0.09416625202758691